## Saif Ad-Din Samir 200346
## سيف الدين سمير شناوي 200346

In [30]:
from typing import List, Dict, Any
from tabulate import tabulate
from anytree import Node, RenderTree

In [31]:
class Token:
    def __init__(self, token_type: str, value: Any = None):
        self.type = token_type
        self.value = value

    def __repr__(self):
        return f"Token(type={self.type}, value={self.value})"

    def __eq__(self, other: Any) -> bool:
        if isinstance(other, Token):
            return self.type == other.type and self.value == other.value
        return False
    
    def __str__(self) -> str:
        return f"Token(type={self.type}, value={self.value})"

In [32]:
class ASTNode:
    def __init__(self, node_type: str, value: Any = None, params: List['ASTNode'] = None, body: List['ASTNode'] = None, name=None):
        self.type = node_type
        self.value = value
        self.params = params if params else []
        self.body = body if body else []
        self.name = name

    def __str__(self, level=0) -> str:
        ret = f"{self.type}({self.name}, {self.value})" if self.value else f"{self.type}({self.name})"
        if self.params or self.body:
            ret += "["
            if self.params:
                ret += " " + ", ".join(param.__str__(level + 1) for param in self.params)
            if self.body:
                if self.params:
                    ret += " | "
                ret += " " + ", ".join(child.__str__(level + 1) for child in self.body)
            ret += "]"
        return ret

In [33]:
def get_current_char(input_str: str, current: int) -> str:
    if current < len(input_str):
        return input_str[current]
    else:
        return None

In [34]:
def tokenizer(input_str: str) -> List[Token]:
    current = 0
    tokens = []
    char_to_token_type = {
        '(': 'paren',
        ')': 'paren'
    }

    while current < len(input_str):
        char = get_current_char(input_str, current)

        if char in char_to_token_type:
            tokens.append(Token(char_to_token_type[char], char))
        elif char.isdigit():
            value = ''
            while char.isdigit():
                value += char
                current += 1
                char = get_current_char(input_str, current)
            tokens.append(Token('number', int(value)))
            continue
        elif char == '"':
            current += 1
            value = ''
            while char != '"':
                value += char
                current += 1
                char = get_current_char(input_str, current)
            tokens.append(Token('string', value))
        elif char.isspace():
            pass 
        elif char.isalpha():
            value = ''
            while char.isalpha():
                value += char
                current += 1
                char = get_current_char(input_str, current)
            tokens.append(Token('name', value))
        else:
            raise TypeError('Unknown character: ' + char)

        current += 1
    return tokens

In [35]:
def parser(tokens: List[Token]) -> ASTNode:
    current = 0

    def parse_node() -> ASTNode:
        nonlocal current
        token = tokens[current]

        if token.type == 'number':
            current += 1
            return ASTNode('NumberLiteral', value=token.value)
        elif token.type == 'string':
            current += 1
            return ASTNode('StringLiteral', value=token.value)
        elif token.type == 'paren' and token.value == '(':
            current += 1
            token = tokens[current]

            node = ASTNode('CallExpression', name=token.value, params=[])

            current += 1 
            while not (token.type == 'paren' and token.value == ')'):
                node.params.append(parse_node())
                token = tokens[current]

            current += 1
            return node
        else:
            raise TypeError(token.type)

    ast = ASTNode('Program', body=[])

    while current < len(tokens):
        ast.body.append(parse_node())

    return ast

In [36]:
def display_tokens(tokens: List[Token]) -> str:
    _table = [(token.type, token.value) for token in tokens]
    headers = ["Type", "Value"]
    table = tabulate(_table, headers=headers, tablefmt="grid")
    print(table)
    return table

In [37]:
def create_anytree_node(ast_node: ASTNode, parent=None) -> Node:
    if ast_node.type == 'NumberLiteral':
        node_name = f"{ast_node.name} ({ast_node.value})" if ast_node.name else f"{ast_node.value}"
    else:
        node_name = f"{ast_node.name}" if ast_node.name else f"{ast_node.type}"
    node = Node(node_name, parent=parent)
    for param in ast_node.params:
        create_anytree_node(param, parent=node)
    for child in ast_node.body:
        create_anytree_node(child, parent=node)
    return node

In [38]:
def display_ast_tree(ast: ASTNode) -> str:
    root = create_anytree_node(ast)
    tree_str = ""
    for pre, _, node in RenderTree(root):
        tree_str += f"{pre}{node.name}\n"
    return tree_str

In [39]:
input_str = '(subtract 4 (subtract 3 1)) (multiply 10 19) (add 4 (subtract 3 (add 4 2)))'
tokens = tokenizer(input_str)
ast = parser(tokens)

In [40]:
display_tokens(tokens)

+--------+----------+
| Type   | Value    |
+========+==========+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 4        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 3        |
+--------+----------+
| number | 1        |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | multiply |
+--------+----------+
| number | 10       |
+--------+----------+
| number | 19       |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | add      |
+--------+----------+
| number | 4        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 3        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   |

'+--------+----------+\n| Type   | Value    |\n+========+==========+\n| paren  | (        |\n+--------+----------+\n| name   | subtract |\n+--------+----------+\n| number | 4        |\n+--------+----------+\n| paren  | (        |\n+--------+----------+\n| name   | subtract |\n+--------+----------+\n| number | 3        |\n+--------+----------+\n| number | 1        |\n+--------+----------+\n| paren  | )        |\n+--------+----------+\n| paren  | )        |\n+--------+----------+\n| paren  | (        |\n+--------+----------+\n| name   | multiply |\n+--------+----------+\n| number | 10       |\n+--------+----------+\n| number | 19       |\n+--------+----------+\n| paren  | )        |\n+--------+----------+\n| paren  | (        |\n+--------+----------+\n| name   | add      |\n+--------+----------+\n| number | 4        |\n+--------+----------+\n| paren  | (        |\n+--------+----------+\n| name   | subtract |\n+--------+----------+\n| number | 3        |\n+--------+----------+\n| paren  |

In [41]:
display_ast_tree(ast)

'Program\n├── subtract\n│   ├── 4\n│   └── subtract\n│       ├── 3\n│       └── 1\n├── multiply\n│   ├── 10\n│   └── 19\n└── add\n    ├── 4\n    └── subtract\n        ├── 3\n        └── add\n            ├── 4\n            └── 2\n'

In [42]:
with open('output.txt', 'w', encoding='utf-8') as f:
    f.write("Tokens:\n")
    f.write(display_tokens(tokens))
    f.write("\n\nAST Tree:\n")
    f.write(display_ast_tree(ast))

+--------+----------+
| Type   | Value    |
+========+==========+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 4        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 3        |
+--------+----------+
| number | 1        |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | multiply |
+--------+----------+
| number | 10       |
+--------+----------+
| number | 19       |
+--------+----------+
| paren  | )        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | add      |
+--------+----------+
| number | 4        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   | subtract |
+--------+----------+
| number | 3        |
+--------+----------+
| paren  | (        |
+--------+----------+
| name   |